In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [ ]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print (x_train.shape)
print (x_test.shape)

In [ ]:
class Autoencoder(Model):
    def __init__(self, input_dim, latent_dim, encoder_activation, decoder_activation):
        super(Autoencoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.encoder_activation = encoder_activation
        self.decoder_activation = decoder_activation

        self.encoder = tf.keras.Sequential([
        # layers.Flatten(),
        layers.Dense(latent_dim, activation=self.encoder_activation),
        ])
        self.decoder = tf.keras.Sequential([
        layers.Dense(784, activation=self.decoder_activation),
        # layers.Reshape((28, 28))
        ])

    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
# declare variables
latent_dim = 64
input_dim = 784

autoencoder = Autoencoder(input_dim, latent_dim, 'relu', 'sigmoid')
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
autoencoder.fit(x_train, x_train, epochs=10, shuffle=True, validation_data=(x_test, x_test))

In [118]:
class SDAE(Model):
    def __init__(self, ae_layers):
        super(SDAE, self).__init__()
        self.ae_layers = ae_layers
        self.models = []

    def make(self):
        for i, layer in enumerate(self.ae_layers[:-1]):
            print('building layer input {} output {}'.format(layer, self.ae_layers[i+1]))
            m = Autoencoder(layer, self.ae_layers[i+1], 'relu', 'sigmoid')
            m.compile(optimizer='adam', loss=losses.MeanSquaredError())
            self.models.append(Autoencoder(layer, self.ae_layers[i+1], 'relu', 'sigmoid'))

    def call(self, train, test, epochs):
        train_set = train
        test_set = test

        print(train_set.shape)
        print(train_set.flatten().shape)

        for m in self.models:
            m.compile(optimizer='adam', loss=losses.MeanSquaredError())
            m.fit(train_set, train_set, epochs=epochs, shuffle=True, validation_data=(test_set, test_set))

            print('before {}'.format(train_set.shape))
            train_set = m.encode(train_set)
            print('after {}'.format(train_set.shape))

            test_set = m.encode(test_set)

In [119]:
ae_layers = [784, 64, 16]

print(x_train.shape)
print(x_test.shape)


x_train = x_train.reshape(60000, 784)
x_test= x_test.reshape(10000, 784)


sdae = SDAE(ae_layers)
sdae.make()
# sdae.compile(optimizer='adam', loss=losses.MeanSquaredError())
sdae.call(x_train, x_test, epochs=2)

(60000, 784)
(10000, 784)
building layer input 784 output 64
building layer input 64 output 16
(60000, 784)
(47040000,)
Epoch 1/2
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0402 - val_loss: 0.0132
Epoch 2/2
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0124 - val_loss: 0.0107
before (60000, 784)
after (60000, 64)
Epoch 1/2


ValueError: in user code:

    /Users/mtigwell/opt/anaconda3/envs/cdl/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/mtigwell/opt/anaconda3/envs/cdl/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/mtigwell/opt/anaconda3/envs/cdl/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/mtigwell/opt/anaconda3/envs/cdl/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/mtigwell/opt/anaconda3/envs/cdl/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args